In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import Perceptron

#####Carregand a base de treino#####

with open("base_treino_secretaria.csv", encoding='utf-8') as f:
    sentencas = f.read().split("\n")

label = []
interacao = []

for sentenca in sentencas[:-1]:
    sentenca = sentenca.split(";")
    label.append(sentenca[0])
    interacao.append(sentenca[1])

####Carregando a base de stop words do PT#####

with open("stop_words.csv", encoding="utf-8") as f:
    pt_stop_words = f.read().split("\n")

####Tratando as features da base de treino####

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words = pt_stop_words)
features = tfidf.fit_transform(interacao).toarray()
labels = label

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(interacao)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#### Instanciando modelos####
NB = MultinomialNB().fit(X_train_tfidf, labels)
#prcpt = Perceptron().fit(X_train_tfidf, labels)
SVM = svm.SVC(kernel='linear', C=0.1).fit(X_train_tfidf, labels)
log = LogisticRegression().fit(X_train_tfidf, labels)

In [2]:
####Avaliando cada Classificador####
from NB import naive_bayes
with open("base_teste_secretaria.csv", encoding="utf-8") as f:
    test_dados = f.read().split("\n")
    
    x_test = []
    y_test = []
    for linha in test_dados[:-1]:
        linha = linha.split(';')
        x_test.append(linha[1])
        y_test.append(linha[0])
        
#y_pred_prcpt =[]
y_pred_SVM = []
y_pred_log = []
y_pred_NB_sk = []
y_pred_NB = []
for x,y in zip(x_test,y_test):
#     prediction = (prcpt.predict(count_vect.transform([x])))
#     y_pred_prcpt.append(prediction[0])
    prediction = (SVM.predict(count_vect.transform([x])))
    y_pred_SVM.append(prediction[0])
    prediction = (log.predict(count_vect.transform([x])))
    y_pred_log.append(prediction[0])
    prediction = (NB.predict(count_vect.transform([x])))
    y_pred_NB_sk.append(prediction[0])
    prediction = (naive_bayes(x))
    y_pred_NB.append(prediction)

In [3]:
import pandas as pd
# df_prcpt = pd.DataFrame()
# df_prcpt['true'] = y_test
# df_prcpt['predicted'] = y_pred_prcpt
df_SVM = pd.DataFrame()
df_SVM['true'] = y_test
df_SVM['predicted'] = y_pred_SVM
df_log = pd.DataFrame()
df_log['true'] = y_test
df_log['predicted'] = y_pred_log
df_NB_sk = pd.DataFrame()
df_NB_sk['true'] = y_test
df_NB_sk['predicted'] = y_pred_NB_sk
df_NB = pd.DataFrame()
df_NB['true'] = y_test
df_NB['predicted'] = y_pred_NB

In [4]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from numpy import mean

#Entrada
# tabela : tabela é um data.frame com strings e o array escroto com colunas com os nomes "predicted" e "true"

def metricas(tabela):
    #replace dos dados
    tabela["predicted"] = tabela["predicted"].apply(lambda x : x.replace("\'",""))
    # pega todos os labels
    labels = tabela.true.unique()
    labels
    precisions = []
    recalls = []
    accuracies = []
    # calcula as métricas para cada label
    for label in labels:
        # binarize
        true_bin = tabela["true"] == label
        predicted_bin = tabela["predicted"] == label
        # calculate metrics
        precisions.append(precision_score(y_true=true_bin, y_pred=predicted_bin))
        recalls.append(recall_score(y_true=true_bin, y_pred=predicted_bin))
        accuracies.append(accuracy_score(y_true=true_bin, y_pred=predicted_bin))
    #print(precisions)
    print("Precisions average: ", mean(precisions))
    print("\nRecalls average: ", mean(recalls))
    print("\nAccuracies average: ", mean(accuracies))

In [5]:
metricas(df_SVM)

Precisions average:  0.6799999999999999

Recalls average:  0.7

Accuracies average:  0.8800000000000001


C:\Users\beatriz.albiero\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [6]:
metricas(df_log)

Precisions average:  0.7

Recalls average:  0.8

Accuracies average:  0.9199999999999999


C:\Users\beatriz.albiero\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
metricas(df_NB_sk)

Precisions average:  0.6799999999999999

Recalls average:  0.7

Accuracies average:  0.8800000000000001


C:\Users\beatriz.albiero\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
metricas(df_NB)

Precisions average:  0.9333333333333332

Recalls average:  0.9

Accuracies average:  0.96


In [9]:
df_NB

,true,predicted
0,contato,contato
1,localizacao,contato
2,horario,horario
3,despedida,despedida
4,despedida,despedida
5,saudacao,saudacao
6,contato,contato
7,despedida,despedida
8,localizacao,localizacao
9,horario,horario
